Подключаем стандартные библиотеки для работы с данными и визуализации данных

In [58]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt #для графики

Импортируем файлик с данными 

In [59]:
data = pd.read_csv('/Users/adeliabiganakova/Desktop/Cancer model/Cancer_Data.csv') 
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

Корректируем данные: убираем значения 'id' 'Unnamed:32', потому что они не информативны и вносят шумы в обучение. Объекты преобразуем в числа, так как дальше мы будем нормализовать данные. Поскольку мы решаем задачу бинарной классификации, в значении diagnosis будет 1, если опухоль злокачественная (M == malignant), и 0, если доброкачественная (B)

In [60]:
data.drop(["Unnamed: 32","id"],axis=1,inplace = True) 
data.diagnosis = [1 if each == "M" else 0 for each in data.diagnosis] 
y = data.diagnosis.values
x_data = data.drop(["diagnosis"],axis = 1)
print(pd.DataFrame(x_data))
print(pd.DataFrame(y))

     radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean   
0          17.99         10.38          122.80     1001.0          0.11840  \
1          20.57         17.77          132.90     1326.0          0.08474   
2          19.69         21.25          130.00     1203.0          0.10960   
3          11.42         20.38           77.58      386.1          0.14250   
4          20.29         14.34          135.10     1297.0          0.10030   
..           ...           ...             ...        ...              ...   
564        21.56         22.39          142.00     1479.0          0.11100   
565        20.13         28.25          131.20     1261.0          0.09780   
566        16.60         28.08          108.30      858.1          0.08455   
567        20.60         29.33          140.10     1265.0          0.11780   
568         7.76         24.54           47.92      181.0          0.05263   

     compactness_mean  concavity_mean  concave points_mean  sym

Нормируем данные 

In [61]:
x = (x_data - np.min(x_data))/(np.max(x_data)-np.min(x_data)) #Normalization Formula
print(pd.DataFrame(x))

     radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean   
0       0.004229      0.002440        0.028867   0.235308         0.000028  \
1       0.004835      0.004177        0.031241   0.311707         0.000020   
2       0.004629      0.004995        0.030559   0.282793         0.000026   
3       0.002685      0.004791        0.018237   0.090762         0.000033   
4       0.004770      0.003371        0.031758   0.304890         0.000024   
..           ...           ...             ...        ...              ...   
564     0.005068      0.005263        0.033380   0.347673         0.000026   
565     0.004732      0.006641        0.030842   0.296427         0.000023   
566     0.003902      0.006601        0.025458   0.201716         0.000020   
567     0.004843      0.006895        0.032934   0.297367         0.000028   
568     0.001824      0.005769        0.011265   0.042548         0.000012   

     compactness_mean  concavity_mean  concave points_mean  sym

Делим все данные на тренировочные и тестирующие

In [62]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

Транспонируем матрицы для того, чтобы можно было их перемножать (ну из-за "nxm mxn")

In [63]:
x_train = x_train.T 
x_test = x_test.T
y_train = y_train.T
y_test = y_test.T

Определяем стандартные веса и байес

In [64]:
def initialize_weights_and_bias(dimension):
    w=np.full((dimension,1),0.01) 
    b = 0.0 
    
    return w,b

В качестве функции выбираем сигмоиду, потому что она точно в диапозоне от 0 до 1, и дифференциируема (можно использовать в градиентном спуске)

In [65]:
def sigmoid(z): 
    y_head = 1/(1+np.exp(-z)) 
    return y_head 

Дальше будут все функции, которые мы используем, но с описанием 